# Conjuntos de datos de entrenamiento, validación y prueba

**Refs**

https://www.geeksforgeeks.org/training-neural-networks-with-validation-using-pytorch/

https://towardsdatascience.com/using-machine-learning-to-predict-fitbit-sleep-scores-496a7d9ec48

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader, Subset, random_split
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt
import numpy as np
import sklearn as skl
from torchviz import make_dot
import torch.optim as optim
from collections import defaultdict

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Usando el dispositivo {}'.format(device))

Consideremos una familia de redes neuronales $y = f_a(x;w_a)$ de distintas arquitecturas $a$.
Aquí, $x$ denota la entrada (ej. features) de la red, $y$ la salida (ej. labels) y $w_a$ los parámetros o pesos sinápticos de la misma.

Redes de distintas arquitecturas pueden pueden aprender datasets de distintas estructuras.
En particular, redes de distintas arquitecturas pueden tener distintos números de parámetros y por ende pueden aprender datasets de distintas complejidades.

Sabemos que redes demasiado simples (con pocos parámetros) no logran aprender datasets suficientemente complejos, y que redes demasiado complejas tienden a sobrefitear datos.
Entonces, nos interesa encontrar aquella red de la familia, que mejor se desempeñe con los datos a disposición, aprendiendo correctamente los datos de entrenamiento, pero también generalizando sin sobrefitear sobre datos de validación.
Para ello, dividimos el conjunto de datos a disposición (el cuál se supone estar compuesto de muestras generadas de manera estadísticamente independiente) en tres conjuntos:

1. el conjunto de entrenamiento (training),

2. el conjunto de validación (validation), y

3. el conjunto de testeo (test).

Luego, para encontrar la red de arquitectura más conveniente, realizamos el siguiente procedimiento para cada arquitectura $a$:

1. Entrenamos la red $f_a(x,w_a)$ optimizando con respecto a $w_a$ sobre las muestras $x$ obtenidas de dataset de entrenamiento, y utilizando la función de pérdida de nuestra preferencia. Esto resulta en valores "optimos" de los parámetros $\hat{w}_a$, de manera que $f_a(x,\hat{w}_a)$ constituye la red entrenada.

2. Luego, usando métricas de nuestra preferencia (ej. la función de pérdida), evaluamos $f_a(x,\hat{w}_a)$ sobre el conjunto de validación, para ver cuán bien generaliza la red ya entrenada.

Luego, elegimos la arquitectura $\hat{a}$ que haya dado los mejores resultados durante el paso de validación 2.
Finalmente, caracterizamos las bondades de nuestra elección $f_{\hat{a}}(x;w_{\hat{a}})$ evaluándola sobre el conjunto de prueba (test).

Veamos un ejemplo con FashionMNIST y una red multicapa de sólo una capa oculta.
Para ello, comenzamos por crear los conjuntos de entrenamiento, validación y testeo.

In [ ]:
# La primera vez esto tarda un rato ya que tiene que bajar los datos de la red.

labels_map = {
    0: "T-Shirt",
    1: "Trouser",
    2: "Pullover",
    3: "Dress",
    4: "Coat",
    5: "Sandal",
    6: "Shirt",
    7: "Sneaker",
    8: "Bag",
    9: "Ankle Boot",
}

train_dataset = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_dataset = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [ ]:
#train = datasets.MNIST('', train = True, transform = transforms, download = True)
#train_dataset,valid_dataset = random_split(train_dataset,[50000,10000])
indices = torch.arange(3000)
train_dataset = Subset(train_dataset,indices)
train_dataset,valid_dataset = random_split(train_dataset,[2000,1000])

In [ ]:
len(train_dataset),len(valid_dataset)

Luego definimos la red neuronal.
Esta es un perceptron con una capa oculta de tamaño arbitrario $n$. 
En este ejemplo, dicho $n$ es el único grado de libertad que dejamos variar, de entre todos los que definen la arquitectura de la red.
En otras palabras, nuestra familia estará compuesta de redes con capas ocultas de distintos tamaños.

In [ ]:
class Net(nn.Module):
    def __init__(self,n=128):
        super(Net,self).__init__()
        self.flatten = nn.Flatten()
        self.relu = nn.ReLU()
        self.linear1 = nn.Linear(28*28,n)
        self.linear2 = nn.Linear(n,n)
        self.linear3 = nn.Linear(n,10)
    def forward(self,x):
        x = self.flatten(x)
        x = self.linear1(x)
        x = self.relu(x)
        x = self.linear2(x)
        x = self.relu(x)
        x = self.linear3(x)
        return x

Implementamos las funciones para entrenar, validar y testear un modelo.

In [ ]:
# Definimos la función de entrenamiento
def train_loop(dataloader,model,loss_fn,optimizer,verbose_each=32):  
    # Calculamos cosas utiles que necesitamos
    num_samples = len(dataloader.dataset)
    # Seteamos el modelo en modo entrenamiento. Esto sirve para activar, por ejemplo, dropout, etc. durante la fase de entrenamiento.
    model.train()
    # Pasamos el modelo la GPU si está disponible.        
    model = model.to(device)    
    # Iteramos sobre lotes (batchs)
    for batch,(X,y) in enumerate(dataloader):
        # Pasamos los tensores a la GPU si está disponible.
        X = X.to(device)
        y = y.to(device)      
        # Calculamos la predicción del modelo y la correspondiente pérdida (error)
        pred = model(X)
        loss = loss_fn(pred,y)
        # Backpropagamos usando el optimizador proveido.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # Imprimimos el progreso cada 100 batchs
        if batch % verbose_each*len(X) == 0:
            loss   = loss.item()
            sample = batch*len(X) # Número de batch * número de muestras en cada batch
            print(f"batch={batch} loss={loss:>7f}  muestras-procesadas:[{sample:>5d}/{num_samples:>5d}]")
            
# De manera similar, definimos la función de validación y testeo
def test_loop(dataloader,model,loss_fn):
    num_samples = len(dataloader.dataset)
    num_batches = len(dataloader)
    avrg_loss    = 0
    frac_correct = 0
    # Seteamos el modelo en modo evaluacion. Esto sirve para desactivar, por ejemplo, dropout, etc. cuando no estamos en una fase de entrenamiento.
    model.eval()
    # Pasamos el modelo la GPU si está disponible.    
    model = model.to(device)    
    # Para validar, desactivamos el cálculo de gradientes.
    with torch.no_grad():
        # Iteramos sobre lotes (batches)
        for X,y in dataloader:
            # Pasamos los tensores a la GPU si está disponible.
            X = X.to(device)
            y = y.to(device)           
            # Calculamos las predicciones del modelo...
            pred = model(X)
            # y las correspondientes pérdidas (errores), los cuales vamos acumulando en un valor total.
            avrg_loss += loss_fn(pred,y).item()
            # También calculamos el número de predicciones correctas, y lo acumulamos en un total.
            frac_correct += (pred.argmax(1)==y).type(torch.float).sum().item()

    # Calculamos la pérdida total y la fracción de clasificaciones correctas, y las imprimimos.
    avrg_loss    /= num_batches
    frac_correct /= num_samples
    print(f"Test Error: \n Accuracy: {frac_correct:>0.5f}, Avg. loss: {avrg_loss:>8f} \n")
    return avrg_loss,frac_correct

Creamos, entrenamos y validamos redes de la familia.

In [ ]:
# Definimos hiperparámetros de entrenamiento
learning_rate = 1e-3
batch_size = 8
num_epochs = 10

# Definimos familia
family = [32,64,128]

# Creamos DataLoader's para cada dataset
train_dataloader = DataLoader(train_dataset, batch_size=batch_size)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size)
test_dataloader  = DataLoader(test_dataset,  batch_size=batch_size)

# Creamos una funcion de perdida
loss_fn = nn.CrossEntropyLoss()

# Iteramos sobre (algunos elementos de) la familia de modelos
performance = {}
for n in family:
    print(f"Model of size n={n}\n-------------------------------")    
    model = Net(n)
    # Creamos un optimizador (un Stochastic Gradient Descent en este caso) para el modelo en cuestion.
    optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)
    # Entrenamos el modelo por varias epocas, registrando al final
    avrg_loss    = None
    frac_correct = None
    performance_n  = {}
    performance[n] = performance_n
    for epoch in range(num_epochs):
        print(f"Epoch={epoch} n={n}\n-------------------------------")
        train_loop(train_dataloader,model,loss_fn,optimizer)
        avrg_loss,frac_correct = test_loop(valid_dataloader,model,loss_fn)
        performance_epoch = {"avrg_loss":avrg_loss,"frac_correct":frac_correct,"model":model}
        performance_n[epoch] = performance_epoch
print("Done!")

Visalicemos para elegir una arquitectura

In [ ]:
# performance

Evaluamos la arquitectura elegida en el dataset de prueba, comparando con lo que da en el dataset de validación

In [ ]:
for n in family:
    chosen = performance[n][max(performance[n].keys())]
    avrg_loss    = chosen["avrg_loss"]
    frac_correct = chosen["frac_correct"]
    model        = chosen["model"]    
    print("------------------------------------------------")
    print(f"n = {n}")
    avrg_loss_prueba,frac_correct_prueba = test_loop(test_dataloader,model,loss_fn)
    print(f"avrg_loss_valid  = {avrg_loss}\t frac_correct_valid  = {frac_correct}")
    print(f"avrg_loss_prueba = {avrg_loss_prueba}\t frac_correct_prueba = {frac_correct_prueba}")